# Experiment
- Hybrid search
- Doc split

Reference: https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [1]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
import sys
from langchain_core.documents.base import Document

sys.path.append("../")
from src.config import Configuration

from collections import Counter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

PATH = '../data/test_major'
docs = pd.read_csv(f"{PATH}/docs.csv")
docs = docs[docs['id'] >= 6]
db = []

for i, data in docs.iterrows():
    doc = Document(page_content=data['content'], metadata={"id": data['id']})
    db.append(doc)

print(len(db))


39


# Hybrid

In [2]:
conf = Configuration()

MODEL = "sentence-transformers/LaBSE"
INDEX = "labse-major"

# MODEL = "keepitreal/vietnamese-sbert"
# INDEX = "vnsbert-major"

WEIGHTS = [0.5, 0.5]
K = 2

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(),
    model_name=MODEL)

In [3]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
conf = Configuration()
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(db)
bm25_retriever.k = K

elastic_vector_search = ElasticsearchStore(
            es_connection=conf.load_elasticsearch_connection(),
            index_name=INDEX,
            embedding=embeddings,
            distance_strategy="EUCLIDEAN_DISTANCE")

es_retriever = elastic_vector_search.as_retriever(search_kwargs={"k": K})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, es_retriever], weights=WEIGHTS
)


# Test

In [9]:
!ls ../data/test_major

docs.csv		    private_test_case.csv
hard_private_test_case.csv  public_test_case.csv


In [4]:
PATH = '../data/test_major'

pv_qas = pd.read_csv(f"{PATH}/private_test_case.csv")
qas = pd.read_csv(f"{PATH}/public_test_case.csv")

def test_major(ensemble_retriever, limit):
    correct = 0
    total = 0
    for id, row in qas.iterrows():
        query = row['public question']
        total += 1
        try:
            res = ensemble_retriever.get_relevant_documents(query)
            for idx, r in enumerate(res):
                target_id = res[0].metadata['id']
                if(int(target_id) == int(row['doc_id'])):
                    correct += 1
                    break
                if idx + 1 == limit:
                    break
        except:
            print(query)
    return [correct,total]

def test_major_private(ensemble_retriever, limit):
    correct = 0
    total = 0
    for id, row in pv_qas.iterrows():
        query = row['private question']
        total += 1
        try:
            res = ensemble_retriever.get_relevant_documents(query)
            for idx, r in enumerate(res):
                target_id = res[0].metadata['id']
                if(int(target_id) == int(row['doc_id'])):
                    correct += 1
                    break
                if idx + 1 == limit:
                    break
        except:
            print(query)
    return [correct,total]

# Benchmark

In [5]:
def write_info(f):
    f.write(f"->\n")
    f.write(f"Model: {MODEL}\n")
    f.write(f"Technique: {MODEL} + Hybrid (full text at BM25)\n")
    f.write(f"Index: {INDEX}\n")
    f.write(f"Params\nweights: {WEIGHTS}, k: {K}\n")

## LaBSE

In [9]:
# done
with open("./results/labse_major_hybrid_bm25full_public.txt", "a") as f:
    top = 1
    # c, t = test_major(ensemble_retriever, top)
    c, t = [1, 2]
    write_info(f)
    f.write(f"Find Top: {top}\n")
    f.write(f"Correct: {str(c)}\n")
    f.write(f"Total: {str(t)}\n")
    f.write(f"Score: {str(c/t)}\n")

In [13]:
with open("./results/labse_major_hybrid_bm25full_public.txt", "a") as f:
    top = 2
    c, t = test_major(ensemble_retriever, top)
    # c, t = [1, 2]
    write_info(f)
    f.write(f"Find Top: {top}\n")
    f.write(f"Correct: {str(c)}\n")
    f.write(f"Total: {str(t)}\n")
    f.write(f"Score: {str(c/t)}\n")

0.7884615384615384

In [10]:
with open("./results/labse_major_hybrid_bm25full_private.txt", "a") as f:
    top = 1
    c, t = test_major_private(ensemble_retriever, 1)
    # c, t = [1, 2]
    write_info(f)
    f.write(f"Find Top: {top}\n")
    f.write(f"Correct: {str(c)}\n")
    f.write(f"Total: {str(t)}\n")
    f.write(f"Score: {str(c/t)}\n")

In [11]:
with open("./results/labse_major_hybrid_bm25full_private.txt", "a") as f:
    top = 2
    c, t = test_major_private(ensemble_retriever, top)
    # c, t = [1, 2]
    write_info(f)
    f.write(f"Find Top: {top}\n")
    f.write(f"Correct: {str(c)}\n")
    f.write(f"Total: {str(t)}\n")
    f.write(f"Score: {str(c/t)}\n")

## VN SBert

In [12]:
a = test_major(ensemble_retriever, 1)
a

Ngành bảo hộ lao động là gì?
Kỹ sư bảo hộ lao động có thể làm việc ở đâu?
Sinh viên ngành bảo hộ lao động được đào tạo những gì?
Điểm nổi bật của chương trình đào tạo ngành bảo hộ lao động là gì?


Năng lực của kỹ sư bảo hộ lao động sau khi tốt nghiệp là gì?

Công nghệ sinh học là gì?


Công nghệ sinh học được ứng dụng trong những lĩnh vực nào?


Sau khi tốt nghiệp ngành Công nghệ sinh học, sinh viên có thể làm những công việc gì?


Chương trình đào tạo ngành Công nghệ sinh học như thế nào?


Cơ hội việc làm ngành Công nghệ sinh học như thế nào?


Tại sao nên học ngành Công nghệ sinh học?


Công tác xã hội là gì?

Người học ngành Công tác xã hội có thể làm việc ở đâu?

Chương trình đào tạo ngành Công tác xã hội gồm những gì?

Sinh viên ngành Công tác xã hội có được thực hành nhiều không?

Sinh viên ngành Công tác xã hội có cơ hội được tiếp xúc với chuyên gia không?

Dược học là gì?
Sinh viên ngành Dược học được trang bị những kiến thức và kỹ năng gì?


Sinh viên ngành Dược học có thể l

0.0

# RUN

In [15]:
query = "Công nghệ thông tin"
results = elasticsearch.similarity_search_with_score(query)
results

[(Document(page_content='Nhà trường đã chủ động triển khai các chương trình hợp tác, liên kết đào tạo với các công ty công nghệ thông tin lớn nhằm hỗ trợ sinh viên về chuyên môn, cũng như tiếp nhận sinh viên thực tập và học tập thực tế tại các công ty, doanh nghiệp. Sinh viên tốt nghiệp ngành Mạng máy tính và truyền thông dữ liệu được đào tạo nắm vững lý thuyết chuyên môn và có khả năng quản lý và vận hành những công nghệ mạng phổ biến như thư tín điện tử, truyền tải tập tin, truyền thông thông tin, hay những công nghệ tiên tiến như điện toán đám mây, tính toán lưới, tính toán di động, xây dựng và vận hành trung tâm dữ liệu, an toàn và bảo mật thông tin, …', metadata={'id': 24.0}),
  0.4918411),
 (Document(page_content='Khoa học máy tính là ngành ứng dụng kiến thức về thuật toán, phương pháp tính toán, ngôn ngữ lập trình, phương pháp phân tích và xây dựng hệ thống thông tin, … để giải quyết các vấn đề thuộc lĩnh vực Khoa học máy tính; phát triển các hệ thống phần mềm máy tính, mô hình 

In [22]:
hybrid_es = ElasticsearchStore(embedding=embeddings,
                          es_connection=config.load_elasticsearch_connection(),
                          index_name="test-basic",
                          strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
                         )

In [24]:
hybrid_es.similarity_search("Tôi muốn học IT")

AuthorizationException: AuthorizationException(403, 'security_exception', 'current license is non-compliant for [Reciprocal Rank Fusion (RRF)]')